In [ ]:
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 transformers==4.39.3 accelerate pandas numpy

In [ ]:
import pandas as pd
from transformers import pipeline

def load_classifier():
    """
    Loads the zero-shot classification model from Hugging Face.

    This is a slow operation and should only be run once.

    Returns:
        A loaded classifier pipeline object.
    """
    print("Loading AI model... (This can take a moment)")
    classifier = pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli"
    )
    print("✅ Model loaded successfully.")
    return classifier

def classify_feedback(feedback_list: list, classifier) -> list:
    """
    Classifies a list of feedback texts using a pre-loaded classifier.

    Args:
        feedback_list (list): A list of strings, where each string is a piece of feedback.
        classifier: The pre-loaded pipeline object from the load_classifier() function.

    Returns:
        list: A list of the top classification labels in the same order as the input.
    """
    if not feedback_list:
        return []

    print(f"Analyzing {len(feedback_list)} feedback entries...")

    # Define the categories for classification
    candidate_labels = ["Top Priority", "Medium Priority", "Least Priority", "feature required"]

    # The pipeline can process a list of texts all at once
    results = classifier(feedback_list, candidate_labels)

    # Extract just the top label for each piece of feedback
    top_priorities = [result['labels'][0] for result in results]

    print("✅ Analysis complete.")
    return top_priorities

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("dataset.csv")
print(df)

                              App_Name        Date      Time   Rating  \
0       EatSure QuickiES Food Delivery  2025-02-27  02:14:38        5   
1       EatSure QuickiES Food Delivery  2025-02-27  01:01:45        1   
2       EatSure QuickiES Food Delivery  2025-02-27  00:40:39        1   
3       EatSure QuickiES Food Delivery  2025-02-27  00:39:58        5   
4       EatSure QuickiES Food Delivery  2025-02-27  00:32:33        5   
...                                ...         ...       ...      ...   
346050  Zomato: Food Delivery & Dining  2024-12-14  00:27:51        5   
346051  Zomato: Food Delivery & Dining  2024-12-14  00:26:43        5   
346052  Zomato: Food Delivery & Dining         NaN       NaN        5   
346053  Zomato: Food Delivery & Dining         NaN       NaN        5   
346054  Zomato: Food Delivery & Dining  2024-12-14  00:07:07        5   

         Comment_Upvotes                                             Review  \
0                      0  I am 1st Time User

In [2]:
# Create a trimmed DataFrame with only the first 1,000 rows
dfd = df.head(1000)

print(f"Original size: {len(df)} rows")
print(f"Trimmed size: {len(dfd)} rows")

Original size: 346055 rows
Trimmed size: 1000 rows


In [ ]:
dfd['Priority'] = np.array(classify_feedback(list(dfd[' Review']), load_classifier()))
print(dfd['Priority'])

Loading AI model... (This can take a moment)


Device set to use cuda:0


✅ Model loaded successfully.
Analyzing 1000 feedback entries...
✅ Analysis complete.
0      feature required
1        Least Priority
2        Least Priority
3      feature required
4       Medium Priority
             ...       
995        Top Priority
996      Least Priority
997     Medium Priority
998    feature required
999    feature required
Name: Priority, Length: 1000, dtype: object


/tmp/ipython-input-2586989546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd['Priority'] = np.array(classify_feedback(list(dfd[' Review']), load_classifier()))


In [ ]:
# Define the name for your new CSV file
output_filename = 'trimmed_feedback.csv'

# Save the DataFrame to the new CSV file
dfd.to_csv(output_filename, index=False)

print(f"✅ Successfully saved {len(dfd)} rows to '{output_filename}'")

✅ Successfully saved 1000 rows to 'trimmed_feedback.csv'
